# 주제 : 뉴스기사 생성 모델 구현하기

## Step 1. 데이터 불러오기 및 전처리

### 문제 01. 필요한 모듈 import

In [1]:
import tensorflow as tf
import numpy as np
import time
import pandas as pd
import os
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 문제 02. 데이터 불러오기

- 한글 뉴스기사 데이터셋
- IT 관련 기사 데이터셋으로 200개의 데이터셋

In [3]:
df = pd.read_csv('https://bit.ly/3n7iHQX')

- 정규 표현식을 사용한 전처리 진행
- 한글, 영어, 숫자를 제외한 모든 문자 제거
- '#' 를 통해 기사 끝 표기를 진행

In [4]:
def clean_sentence(sentence):
    # 한글, 영어, 숫자를 제외한 모든 문자는 제거.
    sentence = re.sub(r'[^0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣 ]',r'', sentence)
    # 문장의 끝 표기.
    sentence += ' #'
    return sentence

In [5]:
clean_sentence('abcef가나다^^$%@12시 땡^^!??')

'abcef가나다12시 땡 #'

In [6]:
df['text'] = df['text'].apply(clean_sentence)

In [7]:
df.head()

,text
0,갤럭시S9 20만 원대 아이폰6S 0원 모비톡 가정의 달 이벤트갤럭시노트8 갤럭시S...
1,LG 그램 100만대 판매기념 한정판 나왔다LG전자가 그램 노트북 누적판매 100만...
2,이게 정말 LG폰이에요G7 씽큐 기분 좋은 스타트20일 서울 신촌역 앞 한 휴대폰 ...
3,애플 10억불vs 삼성 2800만불배상액 종지부 눈앞삼성애플 둥근모서리 디자인특허침...
4,삼성전자 5G 국제 표준 주도한다삼성전자가 5세대5G 이동통신 1차 표준 완성을 위...


In [ ]:
df.shape

(200, 1)

### 문제 03. 데이터 프레임에서 text만 병합하기

`text` 변수에 데이터프레임의 담긴 모든 기사를 join하여 병합.

In [ ]:
text = ' '.join(df['text'])

In [ ]:
# 총 문장의 길이
len(text)

222853

In [ ]:
print(text[:500])

갤럭시S9 20만 원대 아이폰6S 0원 모비톡 가정의 달 이벤트갤럭시노트8 갤럭시S9 갤럭시S8 갤럭시S7 갤럭시S7엣지 아이폰6S 아이폰X 아이폰8 G7 G6 V30 등 다양한 휴대폰 정보가 가득한 스마트폰 공동구매 및 거래 어플 모비톡의 가정의 달 이벤트가 화제다모비톡 단독으로 진행되는 5월 가정의 달 이벤트에 이용자들의 폭발적인 반응이 나타나고 있다 고가의 인기 스마트폰을 파격가에 판매한다는 사실에 각종 커뮤니티와 카페를 중심으로 화제를 모으고 있는 것 특히 갤럭시S9를 20만 원대 아이폰6S는 0원 할부원금을 앞세워 안드로이드와iOS인기 기종을 중심으로 큰 폭의 할인을 펼치는게 주된 요인으로 꼽힌다 모비톡 관계자에 따르면 고마운 사람들에게 감사한 마음을 담아 선물할 기회가 많은 5월 가정의 달을 맞아 공격적인 마케팅을 진행하고 있다며 독보적인 통신비 절약 어플로서 앞으로도 최선을 다하겠다고 밝혔다이 밖에도 모비톡은 갤럭시노트8 V30 구매 시 닌텐도 스위치를 증정한다 스마트폰


### 문제 04. 텍스트 기본 전처리 (preprocessing)

vocabulary 제작.

In [ ]:
vocab = sorted(set(text))

In [ ]:
len(vocab)

1172

In [ ]:
vocab[:20]

[' ',
 '#',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H']

In [ ]:
# 사용자의 입력이 없는 글자를 대상으로 사용하기 위한 목적으로 vocabulary에 추가
vocab.append('?')

### 문제 05. 데이터 형태 변환하기

In [ ]:
#글자 -> index로 변환
char2idx = {u: i for i, u in enumerate(vocab)}

In [ ]:
len(char2idx)

1173

In [ ]:
#index -> 글자로 변환
idx2char = np.array(vocab)

In [ ]:
idx2char

array([' ', '#', '0', ..., '힘', '힙', '?'], dtype='<U1')

## STEP 2. 단어 사전 만들기

### 문제 06. for문을 사용해 문서를 연속된 수치형 값들로 치환.

In [ ]:
text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
text_as_int

array([ 76, 394, 666, ..., 266,   0,   1])

In [ ]:
len(text_as_int)

222853

### 문제 07. 변환 여부 확인. (처음 5개)

In [ ]:
# 원문
text[:5]

'갤럭시S9'

In [ ]:
char2idx['갤'], char2idx['럭'], char2idx['시'], char2idx['S'], char2idx['9']

(76, 394, 666, 30, 11)

In [ ]:
# 변환된 sequence
text_as_int[:5]

array([ 76, 394, 666,  30,  11])

### 문제 08. 각각의 단어사전으로 출력.

In [ ]:
char2idx[' '], char2idx['회'], char2idx['사'], char2idx['#'], char2idx['?'],

(0, 1144, 599, 1, 1172)

## Step 3. 데이터셋 생성 및 EDA

### 문제 09. X, Y 데이터셋 생성하기

In [ ]:
# 단일 입력에 대해 원하는 문장의 최대 길이를 지정.
window_size = 100
shuffle_buffer = 1000
batch_size=128

In [ ]:
# 데이터셋을 만드는 함수를 구현.
def windowed_dataset(series, window_size, shuffle_buffer, batch_size):
    # input data의 텐서 차원 확장 - 가장 오른쪽 텐서에 차원 추가
    series = tf.expand_dims(series, -1)
    # 데이터셋 객체 생성 : 주어진 input data를 통해 데이터셋 생성
    ds = tf.data.Dataset.from_tensor_slices(series)
    # 데이터셋을 주어진 윈도우 크기로 분할 : 시퀀스 데이터셋으로 변경 (n-gram과 유사)
        # drop_remainder=True : 데이터셋을 윈도우 크기로 나눌 때, 나누어지지 않는 윈도우를 제거
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    # 각 윈도우의 데이터셋을 배치 단위로 변환
        # x : 각각의 윈도우, x.batch() : 각 윈도우를 배치로 변환
        # flat_map : 각 윈도우에 적용된 함수의 결과를 평면화하여 하나의 데이터셋으로 변환 (배치로 변환된 윈도우의 데이터가 하나의 데이터셋으로 통합)
    ds = ds.flat_map(lambda x: x.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    # 첫 번째 요소와 마지막 요소를 제거한 데이터의 쌍으로 변환
    ds = ds.map(lambda x: (x[:-1], x[1:]))

    # 데이터셋을 지정된 배치 크기로 묶은 후(.batch()),
    # 미리 로드하여 효율적으로 사용하고 (.prefetch() - 메모리 효율성 제고),
    # 반복하여 모델에 공급할 수 있도록 설정하는 과정 (.repeat())
    return ds.batch(batch_size).prefetch(1).repeat()

In [ ]:
train_data = windowed_dataset(np.array(text_as_int), window_size, shuffle_buffer, batch_size)

### 문제 10. 어휘 사전의 크기

In [ ]:
# 문자로 된 어휘 사전의 크기
vocab_size = len(vocab)
vocab_size

1173

## Step 4.Sequential 모델 구현하기

### 문제 11. keras를 활용한 Sequential 모델 구현.

hyperparamete 설정.

In [ ]:
# 임베딩 차원
embedding_dim = 256

# RNN 유닛(unit) 개수
rnn_units = 1024

In [ ]:
model = tf.keras.Sequential([
    # embedding layer : vector의 length는 window_size로 고정되어 있음
    # vocab size의 데이터를 받아 embdding_dim과 input_lenght에 해당하는 vector로 변환 및 반환
        # embedding layer에는 기본적으로 bias가 존재하지 않음
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=window_size),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         recurrent_initializer='glorot_uniform'),
        # recurrent_initializer : 순환 가중치를 초기화하는 방법 설정
        # glorot_uniform : 무작위로 선택된 작은 값을 사용하여 가중치를 초기화
        # Xavier 초기화의 일종 (초기 가중치를 적절히 설정하여 각 뉴런의 출력 분산을 일정하게 유지하려는 목적으로 사용)
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 256)          300288    
                                                                 
 lstm (LSTM)                 (None, 100, 1024)         5246976   
                                                                 
 dense (Dense)               (None, 100, 1173)         1202325   
                                                                 
Total params: 6749589 (25.75 MB)
Trainable params: 6749589 (25.75 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### 문제 12. 모델을 저장할 Checkpoint 생성.

In [ ]:
checkpoint_path = './models/tmp-checkpoint.h5'
checkpointer = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor='loss',
    verbose=1,
)

### 문제 14. 모델 컴파일

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=0.0001)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['acc'])

### 문제 15. epoch 지정.

**steps_per_epoch에 대하여**


1. fit()함수를 취할 때, 버젼별로 `steps_per_epoch`과, `validation_steps`의 값이 지정되어 있지 않으면 학습이 안되는 현상 존재.

2. 위의 2가지 파라미터에 값을 넣어 주면 정상적으로 학습.

3. `steps_per_epoch` : weight를 업데이트 하는 주기. batch가 다 돌때마다 weight를 업데이트 수행.

4. 'validation_steps' : validation_generator의 weight 업데이트 숫자러. 3번과 사실상 동일.

5. `steps_per_epoch` = len(training_generator)
`validation_steps` = len(validation_generator)로 설정하면 간단.

In [ ]:
steps_per_epoch = (len(text_as_int) - window_size) // (batch_size)
steps_per_epoch

1740

### 문제 16. 모델 학습 이후 callbacks로 앞에서 만든 체크포인트를 할당.

In [ ]:
model.fit(train_data,
          epochs=30,
          steps_per_epoch=steps_per_epoch,
          callbacks=[checkpointer, reduce_lr])

Epoch 1/30
1740/1740 [==============================] - ETA: 0s - loss: 2.6455 - acc: 0.5068
Epoch 1: loss improved from inf to 2.64546, saving model to ./models/tmp-checkpoint.h5
1740/1740 [==============================] - 297s 169ms/step - loss: 2.6455 - acc: 0.5068 - lr: 0.0010
Epoch 2/30


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1740/1740 [==============================] - ETA: 0s - loss: 1.9018 - acc: 0.6304
Epoch 2: loss improved from 2.64546 to 1.90182, saving model to ./models/tmp-checkpoint.h5
1740/1740 [==============================] - 292s 168ms/step - loss: 1.9018 - acc: 0.6304 - lr: 0.0010
Epoch 3/30
1740/1740 [==============================] - ETA: 0s - loss: 1.5417 - acc: 0.6827
Epoch 3: loss improved from 1.90182 to 1.54174, saving model to ./models/tmp-checkpoint.h5
1740/1740 [==============================] - 291s 167ms/step - loss: 1.5417 - acc: 0.6827 - lr: 0.0010
Epoch 4/30
1719/1740 [============================>.] - ETA: 3s - loss: 1.2086 - acc: 0.7369

## Step 5. 모델을 활용한 뉴스기사 생성

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[1, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])

### 문제 17. 저장한 Model Checkpoint를 불러옵니다.

In [ ]:
model.load_weights(checkpoint_path)

### 문제 18. 모델을 build하고 요약 내용을 출력해봅니다.

In [ ]:
model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (1, None, 256)            300288    
_________________________________________________________________
lstm_6 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_6 (Dense)              (1, None, 1173)           1202325   
Total params: 6,749,589
Trainable params: 6,749,589
Non-trainable params: 0
_________________________________________________________________


### 문제 19. 불러온 모델을 활용해 뉴스기사를 생성해봅니다.

In [ ]:
def generate_text(model, start_string):
    # 평가 단계 (학습된 모델을 사용하여 텍스트 생성)

    # 생성할 문자의 수
    num_generate = 1000

    # 시작 문자열을 숫자로 변환(벡터화)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # 여기에서 배치 크기 == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # 배치 차원 제거
        predictions = tf.squeeze(predictions, 0)

        # 범주형 분포를 사용하여 모델에서 리턴한 단어 예측
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # 예측된 단어를 다음 입력으로 모델에 전달
        # 이전 은닉 상태와 함께
        input_eval = tf.expand_dims([predicted_id], 0)
        result_char = idx2char[predicted_id]

        # '#' 문자열을 만나면 종료합니다.
        if result_char == '#':
            break

        text_generated.append(result_char)

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"스마트폰 "))

스마트폰 관계자는 불법 사이트가 마치 정식 웹툰 플랫폼처럼 모든 웹툰을 긁어다가 공개하고 유료로 제공하는 웹툰도 공짜로 풀어서 작품 가치를 헐값으로 만들고 있다면서 지난해 밤토끼와 같은 불법 사이트로 인해 매출이 줄었다 작가들이 공들여 만든 작품이 성인광고로 도배된 사이트에서 미끼상품으로 공개되는 것에 큰 수모를 느끼고 있다고 말했다6일 웹툰 업계에 따르면 현재 한국 웹툰을 불법으로 연재하는 불법 복제 사이트 200여 곳이 운영되고 있다 이 중 가장 큰 불법 사이트는 밤토끼다 지난해 1월부터 본격 운영된 이곳은 네이버카카오레진코믹스짬툰 등 주요 웹툰 1500편을 무단으로 복사해 제공한다 정식 웹툰 플랫폼에 공개된 신작이 두 시간도 안돼 이곳에 올라오는 식이다 밤토끼는 1년 만에 국내 최대 웹툰 플랫폼 네이버를 앞지르기 시작했다 닐슨코리안클릭에 따르면 밤토끼 월 페이지뷰는 1억3709만건지난해 12월 기준으로 네이버1억2081만건를 제쳤다 밤토끼는 국내 경찰의 수사망이 미치지 못하는 해외에 서버를 두고 성매매 성인용품 도박 등 불법 사이트 광고주로부터 수익을 챙기고 있다레진코믹스에 따르면 밤토끼 사이트 운영사는 현재 중앙아메리카 소국 벨리즈로 돼 있다 그러나 이 회사는 우편 사서함 주소만 있는 유령회사이고 인터넷 접속을 제공하는ISP업체는 불가리아에 있는 회사를 사용하며 자료를 저장하는 데이터센터는 우크라이나 업체를 사용한다 레진엔터테인먼트 관계자는 벨리즈 불가리아 업체와 우크라이나 소재 데이터센터에 서비스 차단을 요청했지만 1년 넘게 어떠한 답변도 받지 못했다고 말했다정부도 밤토끼 차단을 시도했지만 번번이 실패했다저작권 권리자가 권리 침해를 신고하면 한국저작권보호원의 심사를 거쳐 방송통신심의원회가ISP업체에 불법 사이트 차단을 요청한다 사이트를 폐쇄할 수는 없지만 국내 이용자 접속을 차단함으로써 직접 피해를 줄이는 방식이다 이때ISP업체가 사용하는 차단 기술은 데이터 신호를 암호화하지 않은 통신 규약 해도적인 재조를 유발하는 실리콘 웨이퍼 시장서를 불법 사용한